In [1]:
!pip install selenium


# Big Data Projekt_Gruppe 4

### Lebensmittelpreisanalyse: Billa, Hofer & Interspar

In diesem Projektabschnitt wurden **Produktdaten aus österreichischen Supermärkten (Billa, Hofer und Interspar)** gesammelt. Fokus lag auf den Kategorien:

- Brot & Gebäck  
- Kühlwaren  
- Fleisch & Fisch  

Mittels **Web Scraping** wurden die Preise und Produktnamen aus den jeweiligen Onlineshops extrahiert und als strukturierte **CSV-Dateien** gespeichert. Diese Dateien dienen als Grundlage für die spätere Analyse, Visualisierung und Vergleichbarkeit der Preise zwischen den Supermärkten.


## Webscraping bei Interspar mit Selenium

In diesem Abschnitt scrapen wir die Produktdaten von **interspar.at** für drei Kategorien:

- Wurst, Fleisch & Fisch
- Kühlregal
- Brot & Gebäck

### Vorgehensweise:
1. Wir rufen jede Kategorie-URL mit **Selenium** im Browser auf.
2. Nach dem Laden parsen wir den HTML-Inhalt zusätzlich mit **BeautifulSoup**.
3. Für jedes Produkt extrahieren wir:
   - Den **Produktnamen**
   - Den **Preis** (bestehend aus zwei `<label>`-Teilen: Euro und Cent)
   - Den **Produktlink**
   - Die **Kategorie**, aus der das Produkt stammt
4. Wir gehen automatisch durch **alle Seiten** innerhalb der Kategorie (via „Nächste Seite“-Button).
5. Abschließend speichern wir alle Daten in der Datei **`interspar_kategorien.csv`**.

**Ergebnis:**  
Eine vollständige Liste aller Produkte aus den drei Kategorien inkl. Preis und Link, ideal zur späteren Analyse und zum Vergleich mit anderen Märkten.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time

# Kategorie-URLs
urls = [
    "https://www.interspar.at/shop/lebensmittel/wurst-fleisch-fisch/c/F3/",
    "https://www.interspar.at/shop/lebensmittel/kuehlregal/c/F2/",
    "https://www.interspar.at/shop/lebensmittel/brot-gebaeck/c/F6/"
]

driver = webdriver.Chrome()
base_url = "https://www.interspar.at"
product_list = []

try:
    for url in urls:
        driver.get(url)
        time.sleep(2)

        while True:
            # Warten auf Produkte
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "productBox"))
            )
            time.sleep(1)

            # HTML einlesen
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            product_boxes = soup.find_all('div', class_='productBox')

            for box in product_boxes:
                link_url = box.get('data-url')
                link = base_url + link_url if link_url else ""

                # Preis zusammensetzen aus zwei <label>
                try:
                    price_container = box.find('div', class_='actualPriceContainer')
                    euros = price_container.find('label', class_='priceInteger').text.strip()
                    cents = price_container.find('label', class_='priceDecimal').text.strip()
                    price = f"{euros},{cents} €"
                except:
                    price = "Kein Preis"

                # Produktname (zweiter Titel ist Name)
                title_tags = box.find_all('div', class_='productTitle')
                if len(title_tags) > 1 and title_tags[1].has_attr('title'):
                    product_name = title_tags[1]['title'].strip()
                elif title_tags and title_tags[0].has_attr('title'):
                    product_name = title_tags[0]['title'].strip()
                else:
                    product_name = "Kein Name"

                product_list.append({
                    'Kategorie': url.split("/")[-3],  # z. B. kuehlregal
                    'Produkt': product_name,
                    'Preis': price,
                    'Link': link
                })

            
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="nächste Seite"]')
                if "disabled" in next_button.get_attribute("class"):
                    break
                next_button.click()
                time.sleep(2)
            except:
                break

finally:
    driver.quit()

# Speichern als CSV
with open('interspar_kategorien.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Kategorie', 'Produkt', 'Preis', 'Link'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte erfolgreich gespeichert in 'interspar_kategorien.csv'")






240 Produkte erfolgreich gespeichert in 'interspar_kategorien.csv'


## Webscraping bei Billa – mehrere Kategorien mit Selenium

Hier nutzen wir **Selenium**, um automatisch Produkte von der Website **shop.billa.at** auszulesen.

### Kategorien, die wir analysieren:
- Kühlwaren
- Brot & Gebäck

### Ablauf:
1. Der Browser ruft nacheinander jede URL der Kategorien auf.
2. Nach dem vollständigen Laden der Produktliste (über `WebDriverWait`) werden alle Produktkarten erfasst.
3. Für jedes Produkt lesen wir:
   - Den **Produktnamen** (aus dem HTML-Attribut `data-teaser-name`)
   - Den **Preis**
4. Zusätzlich speichern wir die jeweilige **Seite/Kategorie**, um die Herkunft nachzuvollziehen.
5. Alle Daten werden in der Datei **`billa_mehrere_seiten.csv`** gespeichert.

**Ergebnis:**  
Eine CSV-Datei mit Produktnamen, Preisen und zugehöriger Kategorie (Seite) – bereit für die weitere Analyse.


In [100]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Liste der Billa-Kategorien
urls = [
    "https://shop.billa.at/kategorie/kuehlwaren-13841",
    "https://shop.billa.at/kategorie/brot-und-gebaeck-13766"
]

driver = webdriver.Chrome()
product_list = []

for url in urls:
    driver.get(url)

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "ws-product-item-base"))
    )
    time.sleep(3)

    products = driver.find_elements(By.CLASS_NAME, "ws-product-item-base")
    print(f"{len(products)} Produkte gefunden auf {url}")

    for product in products:
        # Produktname aus Attribut
        try:
            product_name = product.get_attribute("data-teaser-name")
        except:
            product_name = "Kein Name"

        # Preis extrahieren
        try:
            price = product.find_element(By.CLASS_NAME, "ws-product-price-type__value").text.strip()
        except:
            price = "Kein Preis"

        product_list.append({'Produkt': product_name, 'Preis': price, 'Seite': url})

driver.quit()

# CSV speichern
with open('billa_mehrere_seiten.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Produkt', 'Preis', 'Seite'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte gespeichert.")



30 Produkte gefunden auf https://shop.billa.at/kategorie/kuehlwaren-13841
30 Produkte gefunden auf https://shop.billa.at/kategorie/brot-und-gebaeck-13766
60 Produkte gespeichert.


## Webscraping eines Hofer-Produktbereichs mit Selenium

In diesem Schritt wird mithilfe von **Selenium** automatisiert die Produktübersicht der Kategorie  
**„Fleisch und Fisch“** auf der Website [hofer.at](https://www.hofer.at) ausgelesen.

### Ablauf:
- Der **Browser** wird geöffnet und auf die entsprechende Seite navigiert.
- Es wird gewartet, bis alle Produktkarten vollständig geladen sind.
- Dann werden mit `find_elements` alle Produkte in der Liste durchlaufen.
- Für jedes Produkt erfassen wir:
  - den **Produktnamen**
  - den **Preis**
- Die gesammelten Informationen werden in eine **Liste von Dictionaries** geschrieben.
- Anschließend speichern wir alle Einträge in einer **CSV-Datei namens `hofer_produkte.csv`**.

Ergebnis:  
Die Datei enthält eine strukturierte Liste mit Produktnamen und Preisen aus der Fleisch-und-Fisch-Kategorie von Hofer.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Starte Browser
driver = webdriver.Chrome()
driver.get("https://www.hofer.at/de/sortiment/produktsortiment/fleisch-und-fisch.html")

# Warte bis Produktkarten sichtbar sind
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "item.plp_product"))
)
time.sleep(2)

# Finde alle Produkte
products = driver.find_elements(By.CLASS_NAME, "item.plp_product")
print(f"Gefundene Produkte: {len(products)}")

product_list = []

for product in products:
    # Produktname aus h2
    try:
        name_tag = product.find_element(By.CLASS_NAME, "product-title")
        product_name = name_tag.text.strip()
    except:
        product_name = "Kein Name"

    # Preis aus span
    try:
        price_tag = product.find_element(By.CLASS_NAME, "at-product-price_lbl")
        price = price_tag.text.strip()
    except:
        price = "Kein Preis"

    product_list.append({'Produkt': product_name, 'Preis': price})

driver.quit()

# Export als CSV
with open('hofer_produkte.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Produkt', 'Preis'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte gespeichert in 'hofer_produkte.csv'")


Gefundene Produkte: 100
100 Produkte gespeichert in 'hofer_produkte.csv'


## Webscraping mit Selenium – Hofer

Mit diesem Skript sammeln wir Produktinformationen von der Website **hofer.at**.  
Es werden folgende Produktkategorien automatisiert durchsucht:

- Fleisch und Fisch
- Kühlwaren
- Vorratsschrank

Für jedes Produkt speichern wir:
- **Kategorie**
- **Produktname**
- **Preis**
 
Die gesammelten Daten werden am Ende in der Datei `hofer_kategorien.csv` gespeichert.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Kategorien-URLs
urls = [
    "https://www.hofer.at/de/sortiment/produktsortiment/fleisch-und-fisch.html",
    "https://www.hofer.at/de/sortiment/produktsortiment/kuehlung.html",
    "https://www.hofer.at/de/sortiment/produktsortiment/vorratsschrank.html"
]

# Starte Browser
driver = webdriver.Chrome()
product_list = []

try:
    for url in urls:
        driver.get(url)
        time.sleep(2)

        while True:
            try:
                # Warte auf Produkte
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "item.plp_product"))
                )
                time.sleep(1)

                products = driver.find_elements(By.CLASS_NAME, "item.plp_product")
                print(f"{len(products)} Produkte auf Seite in Kategorie: {url.split('/')[-1]}")

                for product in products:
                    # Produktname
                    try:
                        name_tag = product.find_element(By.CLASS_NAME, "product-title")
                        product_name = name_tag.text.strip()
                    except:
                        product_name = "Kein Name"

                    # Preis
                    try:
                        price_tag = product.find_element(By.CLASS_NAME, "at-product-price_lbl")
                        price = price_tag.text.strip()
                    except:
                        price = "Kein Preis"

                    # Zur Liste hinzufügen
                    product_list.append({
                        'Kategorie': url.split('/')[-1].replace('.html', ''),
                        'Produkt': product_name,
                        'Preis': price
                    })

                # Weiter-Button prüfen
                next_button = driver.find_element(By.CLASS_NAME, "pagination-next")
                if "disabled" in next_button.get_attribute("class"):
                    break
                else:
                    next_button.click()
                    time.sleep(2)

            except:
                break

finally:
    driver.quit()

# Speichere alles als CSV
with open('hofer_kategorien.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Kategorie', 'Produkt', 'Preis'])
    writer.writeheader()
    writer.writerows(product_list)

print(f"{len(product_list)} Produkte aus allen Kategorien gespeichert in 'hofer_kategorien.csv'")



## 🔗 Verbindung zu MongoDB & Datenimport

In diesem Schritt verbinden wir uns mit der lokalen MongoDB-Datenbank und importieren CSV-Dateien mit Produktdaten aus drei Supermärkten:

- **Billa**
- **Hofer**
- **Interspar**

Jede Datei wird als DataFrame geladen, mit dem Namen des Supermarkts versehen und anschließend als JSON-ähnliche Dokumente in die MongoDB-Collection `lebensmittel` geschrieben.

Damit haben wir eine zentrale, NoSQL-basierte Datenspeicherung, auf die wir im weiteren Verlauf flexibel zugreifen können.


In [11]:
!pip install pandas pymongo

In [21]:
import pandas as pd
from pymongo import MongoClient

# Verbindung zur MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["supermarkt_preise"]
collection = db["lebensmittel"]

# Optional: Bestehende Daten vorher löschen
collection.delete_many({})

# Deine Dateien + zugehöriger Supermarktname
files_with_market = {
    "../data/billa_mehrere_seiten.csv": "Billa",
    "../data/hofer_kategorien.csv": "Hofer",
    "../data/interspar_kategorien.csv": "Interspar"
}

# Alle in dieselbe Collection importieren
for filepath, supermarkt in files_with_market.items():
    try:
        df = pd.read_csv(filepath)
        df["Supermarkt"] = supermarkt  
        data = df.to_dict(orient="records")
        collection.insert_many(data)
        print(f"{len(data)} Datensätze aus '{supermarkt}' importiert.")
    except Exception as e:
        print(f"Fehler bei Datei {filepath}: {e}")


60 Datensätze aus 'Billa' importiert.
300 Datensätze aus 'Hofer' importiert.
240 Datensätze aus 'Interspar' importiert.


## Fazit – Datensammlung per Webscraping

Wir haben erfolgreich Produktdaten aus drei großen österreichischen Supermärkten gesammelt:

- **Billa** (über Kategorie-URLs)
- **Hofer** (inkl. Navigation durch Seiten)
- **Interspar** (kombiniert Selenium + BeautifulSoup)

Dabei wurden folgende Informationen extrahiert und gespeichert:
- Produktname
- Preis
- Kategorie
- Link (Interspar)

Die gesammelten Daten wurden als **CSV-Dateien** exportiert und stehen nun für weitere Analysen bereit – z. B. Preisvergleiche oder MapReduce-Berechnungen.

**Ergebnis:**  
Wir verfügen nun über eine konsistente und strukturierte Datengrundlage aus mehreren Quellen, die sich für Big-Data-Analysen eignet.
